# Trace Recording Example

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

In [ ]:
from trulens_eval.trace import Tracer, TracerProvider, tracer_provider
tp = tracer_provider

with tp.trace() as tracer:
    with tracer.method() as span1:
        print(type(span1), span1)

    with tp.trace() as inner_tracer:

        with tracer.method() as span2:
            print(type(span2), span2)
            raise Exception("test outer tracer error")

        with inner_tracer.method() as span3:
            print(type(span3), span3)
            raise Exception("test inner tracer error")

In [ ]:
tracer.spans

In [ ]:
inner_tracer.spans

In [ ]:
from trulens_eval.trace import Tracer, TracerProvider, tracer_provider, get_tracer

tracer = get_tracer()
inner_tracer = get_tracer()

with tracer.method() as span1:
    print(type(span1), span1)

    with tracer.method() as span2:
        print(type(span2), span2)

    with inner_tracer.method() as span3:
        print(type(span3), span3)

In [ ]:
from concurrent.futures import as_completed
from time import sleep

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
from tqdm.auto import tqdm

from trulens_eval import Feedback
from trulens_eval import Tru
from trulens_eval.feedback.provider.hugs import Dummy
from trulens_eval.schema.feedback import FeedbackMode
from trulens_eval.tru_custom_app import TruCustomApp
from trulens_eval.utils.threading import TP

tp = TP()

d = Dummy(
    loading_prob=0.0,
    freeze_prob=0.0, # we expect requests to have their own timeouts so freeze should never happen
    error_prob=0.0,
    overloaded_prob=0.0,
    rpm=1000,
    alloc = 0, # how much fake data to allocate during requests
    delay = 10.0
)

tru = Tru()

#tru.reset_database()

tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.parent.resolve()
)

In [ ]:
# Create custom app:
ca = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [ ]:
with ta as recorder:
    res = ca.respond_to_query(f"hello")

rec = recorder.get()
assert rec is not None

In [ ]:
get_tracer().spans

In [ ]:
recorder.tracer[1].spans